In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import tkinter as tk
from tkinter import filedialog

def image_file_counter(path):
    files = 0
    for _, _, filenames in os.walk(path):
        files += len(filenames)
    return files + 1

def spatial_filtering(depth_frame, magnitude=2, alpha=0.5, delta=20, holes_fill=0):
    spatial = rs.spatial_filter()
    spatial.set_option(rs.option.filter_magnitude, magnitude)
    spatial.set_option(rs.option.filter_smooth_alpha, alpha)
    spatial.set_option(rs.option.filter_smooth_delta, delta)
    spatial.set_option(rs.option.holes_fill, holes_fill)
    depth_frame = spatial.process(depth_frame)
    return depth_frame

def hole_filling(depth_frame):
    hole_filling = rs.hole_filling_filter()
    depth_frame = hole_filling.process(depth_frame)
    return depth_frame

# Define global variables
rgb_img_path = 'captured_images/rgb_image/'
depth_img_path = 'captured_images/depth_image/'
colored_depth_img_path = 'captured_images/coloured_depth_image/'
intrinsics = True
rotate_camera = False

# Create a Tkinter window
root = tk.Tk()
root.title("Intel Realsense D435 GUI")

# Function to save images
def save_images():
    img_counter = image_file_counter(rgb_img_path)

    # Create stream folders if they don't exist
    if not os.path.exists(rgb_img_path):
        os.makedirs(rgb_img_path)
    if not os.path.exists(depth_img_path):
        os.makedirs(depth_img_path)
    if not os.path.exists(colored_depth_img_path):
        os.makedirs(colored_depth_img_path)

    filename = str(img_counter) + '.png'
    filename_raw = str(img_counter) + '.raw'

    # Save the rgb colour image
    cv2.imwrite(os.path.join(rgb_img_path, filename), color_image)
    # Save the depth image in raw binary format uint16.
    f = open(os.path.join(depth_img_path, filename_raw), mode='wb')
    depth_image.tofile(f)
    cv2.imwrite(os.path.join(colored_depth_img_path, filename), depth_color_image)

    print('Images have been successfully saved')

# Create a "Save" button
save_button = tk.Button(root, text="Save", command=save_images)
save_button.pack(pady=10)

# Configure all streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start the streaming
print("Starting up the Intel Realsense D435...")
print("")
profile = pipeline.start(config)

# Get the depth sensor's depth scale
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: ", depth_scale)
print("")

emitter = depth_sensor.get_option(rs.option.emitter_enabled)
print("emitter = ", emitter)
set_emitter = 1
depth_sensor.set_option(rs.option.emitter_enabled, set_emitter)
emitter1 = depth_sensor.get_option(rs.option.emitter_enabled)
print("new emitter = ", emitter1)

# Create an align object to align depth image to color image
align_to = rs.stream.color
align = rs.align(align_to)

try:
    # Skip the first 30 frames for Auto-Exposure adjustment
    for x in range(30):
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)

    print("Intel Realsense D435 started successfully.")
    print("")

    while True:
        # Wait for a coherent pair of frames
        frames = pipeline.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        if intrinsics:
            print("Intel Realsense D435 Camera Intrinsics: ")
            print("========================================")
            print(depth_frame.profile.as_video_stream_profile().intrinsics)
            print(color_frame.profile.as_video_stream_profile().intrinsics)
            print("")
            intrinsics = False

        # Apply filtering to the depth image
        depth_frame = spatial_filtering(depth_frame, magnitude=2, alpha=0.5, delta=50, holes_fill=0)
        depth_frame = hole_filling(depth_frame)

        # Colourise the depth map
        depth_color_frame = rs.colorizer().colorize(depth_frame)

        # Convert images to numpy arrays
        depth_image = np.array(depth_frame.get_data())
        depth_color_image = np.asanyarray(depth_color_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Rotate images if the realsense camera is placed vertically
        if rotate_camera:
            depth_image = np.rot90(depth_image)
            depth_color_image = np.rot90(depth_color_image)
            color_image = np.rot90(color_image)

        # Stack rgb and depth map images horizontally for visualisation only
        images = np.hstack((color_image, depth_color_image))

        # Show horizontally stacked rgb and depth map images
        cv2.namedWindow('RGB and Depth Map Images')
        cv2.imshow('RGB and Depth Map Images', images)
        c = cv2.waitKey(1)

        # If the 'q' key or Esc is pressed, exit the loop and destroy the Tkinter window
        if c == ord('q') or c == 27:
            break

        # Update the Tkinter window
        root.update()

finally:
    # Stop streaming
    pipeline.stop()

# Close the Tkinter window
root.destroy()
cv2.destroyAllWindows()